In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Use CUDA</h2>

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
import numpy as np
import pandas as pd
from scipy import optimize, special

F = np.array([[1.0000, 0.1000],
              [0, 1.0000]])

G = np.array([[0.0050],
              [0.1000]])

C = np.array([[1, 0]])

D = 0
Gain = np.array([16.0302, 5.6622])

L = np.array([[0.9902],
              [0.9892]])
safex = [26,31]

In [5]:
k=14
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
X = np.zeros((2, input_len + 1))
Xe = np.zeros((2, input_len + 1))
e = np.zeros((2, input_len))
U = np.zeros((1,input_len))
Y = np.zeros((1,input_len))
r = np.zeros((1,input_len))
z = np.zeros((1,input_len))
S = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])      
n2 = np.random.normal(0,0.001, [2, input_len])

In [6]:
for i in range(input_len):
    e[:, i] = X[:, i] - Xe[:, i]
    U[:,i] = -(Gain @ Xe[:, i])
    X[:, i+1] = F @ X[:, i] + G @ U[:, i] + n2[:, i]
    Y[:,i] = C @ X[:, i] + n1[:,i]
    r[:,i] = Y[:,i] - C @ Xe[:, i]
    Xe[:, i+1] = F @ Xe[:, i] + G @ U[:,i] + L @ r[:,i]
covn_r=np.cov(r)

In [7]:
attackK= [0, 0.04155838 , 0.63838859 , 1.75426837 ,2.44935936 , 3.6651983 , 5.16561739 , 6.95368618 , 9.11716362 ,11.67267658 ,14.29706254, 17.41085821  ,0.       ,   0.    ]
k=len(attackK)
attack2K = [-30.971675   ,-30.971675  ,  -1.11684483 ,-30.90514398, -28.29714574 , -28.61888554, -28.91105434, -29.17567371, -29.41469648, -29.21724115 , -28.49741602 , 30.971675,     0.     ,      0. ]
k1=len(attack2K)

In [8]:
inputs = np.random.normal(0, 1, k)
input_len=len(inputs)
Xk = np.zeros((2, input_len + 1))
Xek = np.zeros((2, input_len + 1))
ek = np.zeros((2, input_len))
Uk = np.zeros((1,input_len))
Uak = np.zeros((1,input_len))
Yk = np.zeros((1,input_len))
rk = np.zeros((1,input_len))
covn_eK = np.zeros((2, 2))
covn_rK = np.zeros((2, 2))
zk = np.zeros((1,input_len))
Sk = np.zeros((input_len))
n1 = np.random.normal(0,0.01, [1, input_len])      
n2 = np.random.normal(0,0.001, [2,input_len])

In [9]:
residuek=[]
mzk=[]
bias=4
for i in range(input_len):
    ek[:, i] = Xk[:, i] - Xek[:, i]
    Uk[:,i] = -(Gain @ Xek[:, i])
    Uak[:,i] = -(Gain @ Xek[:, i])+attack2K[i]
    Xk[:, i+1] = F @ Xk[:, i] + G @ Uak[:, i] + n2[:, i]
    Yk[:,i] = C @ Xk[:, i] + n1[:,i] + attackK[i]
    rk[:,i] = Yk[:,i] - C @ Xek[:, i]
    Xek[:, i+1] = F @ Xek[:, i] + G @ Uk[:,i] + L @ rk[:,i]
    zk[:,i] = rk[:,i].T * covn_r * rk[:,i]
    residuek.append(rk[:,i])
    mzk.append(zk[:,i])
    Sk[i] = max((Sk[i-1] + zk[:,i] - bias), 0)

In [10]:
Sk

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
m=np.mean(np.array(mzk))

In [12]:
import math
def discrete_frame(Sk,N,th):
        delSi = 2*th/(2*N-1)
        Ek=np.zeros((len(Sk)))
        for i in range(len(Sk)):
            if Sk[i]<delSi[i]/2:
                Ek[i]=0
            elif Sk[i]>=th[i]:
                Ek[i]=N
            else :
                Ek[i]=math.ceil(math.floor(Sk[i]/(delSi[i]/2))/2)
        return Ek

In [13]:
N=10

In [14]:
residuek=np.absolute(np.array(residuek))

In [15]:
r_mean=np.array([])
for i in range(len(residuek)):
    sums=0
    for j in range(i+1):
        sums+=residuek[j]
    mean=sums/(j+1)
    r_mean=np.append(r_mean,mean)

In [16]:
r_mean

array([0.00345009, 0.05337163, 0.07635245, 0.18199853, 0.16500704,
       0.15223018, 0.14443103, 0.1386415 , 0.13745976, 0.1499943 ,
       0.13696959, 0.14167598, 1.69303926, 1.78586384])

In [17]:
np.count_nonzero(r_mean>0.01)

13

In [18]:
import numpy as np
from scipy.special import erf
import scipy.special as special
from scipy import special, optimize
def markov_transition(N, th, bias):
    temp = np.zeros((N+1, N+1))
    for i in range(1,N+1):
        for j in range(1,N+1):
            if j == 1 & i <= N:
                temp[i, j] = Tx(1-i, th, bias, N)
            elif j > 1 & j < N+1 & i <= N:
                temp[i, j] = Px(j-i, th, bias, N)
            elif j == N+1 & i<=N:
                temp[i, j] = 1 - Tx(N-i, th, bias, N)
            elif i == N+1 & j < N+1:
                temp[i, j] = 0
            else:
                temp[i, j] = 1
    P = temp
    return P

def Px(x, th, bias, N):
    delSi = 2*th/(2*N-1)
    z1 = np.max(x*delSi + 0.5*delSi + bias, 0)
    p1 = special.gammainc(m/2, z1/2)
    z2 = np.max(x*delSi - 0.5*delSi + bias, 0)
    p2 = special.gammainc(m/2, z2/2)
    p = p1-p2
    return p
def Tx(x, th, bias, N):
    delSi = 2*th/(2*N-1)
    z = np.max(x*delSi + 0.5*delSi + bias, 0)
    t = special.gammainc(m/2, z/2)
    return t

In [20]:
class Optimization():
    
    def __init__(self,params,m,N,Ek):
        self.params=params.T
        self.threshold=params[:,0]
        self.bias=params[:,1]
        self.m=m
        self.N=N
        self.Ek=Ek

    def gradient(self):
        state=self.Ek
        x=np.zeros((len(state)))
        z=np.zeros((len(state)))
        t=np.zeros((len(state)))
        dt_dth=np.zeros((len(state)))
        dt_dbias=np.zeros((len(state)))
        for i in range(len(state)):
            x[i]=self.N-1-state[i]
            delSi = 2 * self.threshold[i] / (2 * self.N - 1)
            z[i] = np.maximum(x[i] * delSi + 0.5 * delSi + self.bias[i], 0)
            t[i] = 1 - special.gammainc(m / 2, z[i]/ 2)
            dt_dth[i] = 0.5 * z[i] * np.exp(-z[i] / 2) * x[i] / (2 *self.N - 1)
            dt_dbias[i] = 0.5 * z[i] * np.exp(-z[i] / 2)
        return np.array([-dt_dth, -dt_dbias])

    def adam_optimizer(self,learning_rate=0.8, beta1=0.9, beta2=0.999, epsilon=1e-8, num_iterations=2):
        p = np.zeros_like(self.params)
        v = np.zeros_like(self.params)

        for t in range(1,num_iterations+1):
            gradient = self.gradient()
            p = beta1 * p + (1 - beta1) * gradient
            v = beta2 * v + (1 - beta2) * (gradient ** 2)
            p_hat = p / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)
            self.params += learning_rate * p_hat / (np.sqrt(v_hat) + epsilon)

        return self.params

In [21]:
import numpy as np

class RLEnvironment():
    def __init__(self, num_states,cusum,residual,action,N):
        self.num_states = num_states
        self.cusum = cusum
        self.next_cusum=cusum
        self.residual = residual
        self.threshold = action[:,0]
        self.bias=action[:,1]
        self.absorb=N
        self.current_state = None
        
    def discrete_frame(self):
        delSi = 2*self.threshold/(2*self.absorb-1)
        Ek=np.zeros((len(self.next_cusum)))
        for i in range(len(self.next_cusum)):
            if self.next_cusum[i]<delSi[i]/2:
                Ek[i]=0
            elif self.next_cusum[i]>=self.threshold[i]:
                Ek[i]=self.absorb
            else :
                Ek[i]=math.ceil(math.floor(self.next_cusum[i]/(delSi[i]/2))/2)
        return Ek
        
    def reset(self):
        self.current_state = self.discrete_frame()
        return self.current_state
    
    def calculate_reward(self,i):
        # Calculate reward based on attack detection condition
        if self.residual[i]>0.01 and self.cusum[i]> self.threshold[i]:
            return 10
        elif self.residual[i]<0.01 and self.cusum[i]<self.threshold[i]:
            return 1
        return -10

    def step(self):
        self.next_cusum=self.cusum-self.bias
        next_state = self.discrete_frame()
        return next_state,self.next_cusum

In [22]:
# Hyperparameters
N=10
learning_rate=0.5
threshold=np.ones((len(Sk),1))*6
bias=np.ones((len(Sk),1))*4
action = np.concatenate((threshold,bias),axis=1)
Ek=discrete_frame(Sk,N,threshold)
optimizer=Optimization(action,m,N,Ek)
env = RLEnvironment(len(Sk),Sk,r_mean,action,N)
num_episodes=10
Q_values = np.zeros((num_episodes, len(Ek)))
final_action=[]

In [23]:
for episode in range(num_episodes):
    state = env.reset()  # Reset the environment
    for i in range(len(Sk)):
        
        reward = env.calculate_reward(i)
        
        if episode>0:
            Q_values[episode, i] = reward + learning_rate*Q_values[episode-1,i]S
        
        
    next_state,next_cusum = env.step()
    action=optimizer.adam_optimizer()
    final_action.append(np.copy(action))
    optimizer.Ek=next_state


In [24]:
final_action[np.argmax(np.mean(Q_values,axis=1))]#optimal threshold and bias array for each cusum value

array([[-0.09801163, -0.09801163, -0.09801163, -0.09801163, -0.09801163,
        -0.09801163, -0.09801163, -0.09801163, -0.09801163, -0.09801163,
        -0.09801163, -0.09801163, -0.09801163, -0.09801163],
       [-2.09801235, -2.09801235, -2.09801235, -2.09801235, -2.09801235,
        -2.09801235, -2.09801235, -2.09801235, -2.09801235, -2.09801235,
        -2.09801235, -2.09801235, -2.09801235, -2.09801235]])

In [25]:
final_action

[array([[4.42227368, 4.42227368, 4.42227368, 4.42227368, 4.42227368,
         4.42227368, 4.42227368, 4.42227368, 4.42227368, 4.42227368,
         4.42227368, 4.42227368, 4.42227368, 4.42227368],
        [2.42227325, 2.42227325, 2.42227325, 2.42227325, 2.42227325,
         2.42227325, 2.42227325, 2.42227325, 2.42227325, 2.42227325,
         2.42227325, 2.42227325, 2.42227325, 2.42227325]]),
 array([[2.83762332, 2.83762332, 2.83762332, 2.83762332, 2.83762332,
         2.83762332, 2.83762332, 2.83762332, 2.83762332, 2.83762332,
         2.83762332, 2.83762332, 2.83762332, 2.83762332],
        [0.83762275, 0.83762275, 0.83762275, 0.83762275, 0.83762275,
         0.83762275, 0.83762275, 0.83762275, 0.83762275, 0.83762275,
         0.83762275, 0.83762275, 0.83762275, 0.83762275]]),
 array([[ 1.2380348 ,  1.2380348 ,  1.2380348 ,  1.2380348 ,  1.2380348 ,
          1.2380348 ,  1.2380348 ,  1.2380348 ,  1.2380348 ,  1.2380348 ,
          1.2380348 ,  1.2380348 ,  1.2380348 ,  1.2380348 ],
  